<a href="https://colab.research.google.com/github/amriikk/smartphone-teens/blob/main/HW2_group.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt


try:
    df_Teens = pd.read_csv('/content/drive/MyDrive/teen_phone_addiction_dataset.csv')
    print("SUCCESS: 'data' loaded!")
except FileNotFoundError:
    print("ERROR: Could not find 'baseball.csv'. Please upload it.")


SUCCESS: 'data' loaded!


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/teen_phone_addiction_dataset.csv')

# Drop non-predictive columns (ID and Name) and high-cardinality columns (Location)
# 'Location' has ~2700 unique values for 3000 rows, leading to overfitting.
df_clean = df.drop(columns=['ID', 'Name', 'Location'])

# Define targets
targets = ['Anxiety_Level', 'Depression_Level']

# Convert categorical variables into dummy/indicator variables
df_encoded = pd.get_dummies(df_clean, columns=['Gender', 'School_Grade', 'Phone_Usage_Purpose'], drop_first=True)

results = []

for target in targets:
    # 1. Individual R-squared (Numeric features only)
    numeric_features = [col for col in df_clean.select_dtypes(include=['number']).columns if col not in targets]

    for feat in numeric_features:
        X = df_clean[[feat]]
        y = df_clean[target]
        model = LinearRegression().fit(X, y)
        r2 = model.score(X, y)
        results.append({'Target': target, 'Feature': feat, 'R2_Score': r2})

    # 2. Combined R-squared (All meaningful features including categorical)
    X_all = df_encoded.drop(columns=targets)
    y_all = df_encoded[target]
    model_all = LinearRegression().fit(X_all, y_all)
    r2_all = model_all.score(X_all, y_all)
    results.append({'Target': target, 'Feature': 'COMBINED_MODEL (All Features)', 'R2_Score': r2_all})

# Create and display results table
r2_df = pd.DataFrame(results)
r2_df = r2_df.sort_values(by=['Target', 'R2_Score'], ascending=[True, False])

print(r2_df)
r2_df.to_csv('rsquared_results.csv', index=False)

              Target                        Feature  R2_Score
17     Anxiety_Level  COMBINED_MODEL (All Features)  0.007224
13     Anxiety_Level              Time_on_Education  0.000607
9      Anxiety_Level           Phone_Checks_Per_Day  0.000324
16     Anxiety_Level                Addiction_Level  0.000256
12     Anxiety_Level                 Time_on_Gaming  0.000215
0      Anxiety_Level                            Age  0.000215
7      Anxiety_Level               Parental_Control  0.000133
14     Anxiety_Level           Family_Communication  0.000123
2      Anxiety_Level                    Sleep_Hours  0.000097
10     Anxiety_Level                Apps_Used_Daily  0.000055
1      Anxiety_Level              Daily_Usage_Hours  0.000052
15     Anxiety_Level            Weekend_Usage_Hours  0.000045
8      Anxiety_Level         Screen_Time_Before_Bed  0.000022
5      Anxiety_Level                 Exercise_Hours  0.000020
4      Anxiety_Level            Social_Interactions  0.000019
6      A

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load and clean
df = pd.read_csv('/content/drive/MyDrive/teen_phone_addiction_dataset.csv')
df_clean = df.drop(columns=['ID', 'Name', 'Location'])
df_encoded = pd.get_dummies(df_clean, columns=['Gender', 'School_Grade', 'Phone_Usage_Purpose'], drop_first=True)

# Define feature groups
groups = {
    "Usage_Metrics": ['Daily_Usage_Hours', 'Phone_Checks_Per_Day', 'Apps_Used_Daily', 'Time_on_Social_Media', 'Time_on_Gaming', 'Time_on_Education', 'Weekend_Usage_Hours', 'Screen_Time_Before_Bed', 'Addiction_Level'],
    "Lifestyle_Health": ['Sleep_Hours', 'Exercise_Hours', 'Social_Interactions', 'Family_Communication'],
    "Academic_Personal": ['Academic_Performance', 'Self_Esteem'],
    "Demographics": ['Age', 'Gender_Male', 'Gender_Other'],
    "Usage_Purpose": [col for col in df_encoded.columns if 'Phone_Usage_Purpose' in col]
}

results = []
for target in ['Anxiety_Level', 'Depression_Level']:
    for group_name, features in groups.items():
        X = df_encoded[features]
        y = df_encoded[target]
        r2 = LinearRegression().fit(X, y).score(X, y)
        results.append({'Target': target, 'Group': group_name, 'R2': r2})

# Convert to DataFrame for viewing
results_df = pd.DataFrame(results)
print(results_df.sort_values(by='R2', ascending=False))

             Target              Group        R2
8  Depression_Level       Demographics  0.003483
4     Anxiety_Level      Usage_Purpose  0.002595
5  Depression_Level      Usage_Metrics  0.002161
0     Anxiety_Level      Usage_Metrics  0.001761
7  Depression_Level  Academic_Personal  0.001421
6  Depression_Level   Lifestyle_Health  0.001019
9  Depression_Level      Usage_Purpose  0.000459
3     Anxiety_Level       Demographics  0.000306
1     Anxiety_Level   Lifestyle_Health  0.000265
2     Anxiety_Level  Academic_Personal  0.000028


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/teen_phone_addiction_dataset.csv')

# Target variable
target = 'Addiction_Level'

# Drop non-predictive columns
cols_to_drop = ['ID', 'Name', 'Location', target]
features_all = [col for col in df.columns if col not in cols_to_drop]

results = []

# 1. Individual R-squared for each variable
for feature in features_all:
    if df[feature].dtype in ['int64', 'float64']:
        # Numeric Variable
        X = df[[feature]].values
        y = df[target].values
    else:
        # Categorical Variable (Gender, Purpose, etc.)
        X = pd.get_dummies(df[[feature]], drop_first=True)
        y = df[target].values

    model = LinearRegression().fit(X, y)
    results.append({'Variable': feature, 'R2_Score': model.score(X, y)})

# 2. Combined R-squared for all meaningful variables
df_clean = df.drop(columns=['ID', 'Name', 'Location'])
df_encoded = pd.get_dummies(df_clean, columns=['Gender', 'School_Grade', 'Phone_Usage_Purpose'], drop_first=True)
X_all = df_encoded.drop(columns=[target])
y_all = df_encoded[target]

combined_r2 = LinearRegression().fit(X_all, y_all).score(X_all, y_all)
results.append({'Variable': 'ALL_VARIABLES_COMBINED', 'R2_Score': combined_r2})

# Output results
addiction_r2_df = pd.DataFrame(results).sort_values(by='R2_Score', ascending=False)
print(addiction_r2_df)

                  Variable      R2_Score
21  ALL_VARIABLES_COMBINED  7.412440e-01
3        Daily_Usage_Hours  3.609253e-01
14         Apps_Used_Daily  1.019442e-01
15    Time_on_Social_Media  9.399019e-02
16          Time_on_Gaming  7.456155e-02
13    Phone_Checks_Per_Day  6.068436e-02
4              Sleep_Hours  4.695068e-02
18     Phone_Usage_Purpose  2.086913e-03
2             School_Grade  1.567418e-03
1                   Gender  1.030365e-03
0                      Age  9.800764e-04
10             Self_Esteem  4.969178e-04
7           Exercise_Hours  4.416251e-04
8            Anxiety_Level  2.561735e-04
12  Screen_Time_Before_Bed  1.900048e-04
20     Weekend_Usage_Hours  1.702750e-04
5     Academic_Performance  1.504159e-04
6      Social_Interactions  1.130084e-04
19    Family_Communication  1.098676e-04
9         Depression_Level  7.209436e-05
11        Parental_Control  1.031789e-06
17       Time_on_Education  3.430654e-07


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/teen_phone_addiction_dataset.csv')

# Define independent variables (X) and target variable (y)
X_features = [
    'Daily_Usage_Hours',
    'Apps_Used_Daily',
    'Time_on_Social_Media',
    'Time_on_Gaming',
    'Phone_Checks_Per_Day'
]
target = 'Addiction_Level'

X = df[X_features]
y = df[target]

# Initialize and fit the Multiple Linear Regression model
model = LinearRegression()
model.fit(X, y)

# Calculate and print the R-squared value
r_squared = model.score(X, y)
print(f"R-squared: {r_squared:.4f}")

R-squared: 0.6855
